In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jun 21 04:00:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# 데이터 다운로드 및 불러오기


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/datacap/final

/content/drive/MyDrive/Colab Notebooks/datacap/final


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datacap/final/Cleand_newsdata.csv')

In [ ]:
data

,Unnamed: 0,Headline,Short
0,0,exbank official booked for cheating bank of c...,the cbi on saturday booked four former officia...
1,1,suprtheme court to go paperless in month cji,chief justice js khehar ha said the suprtheme ...
2,2,at least killed injured in blast in sylhet ...,at least three people were killed including a...
3,3,why ha reliance been barred from trading in fu...,mukesh ambaniled reliance industries ril wa ...
4,4,was stopped from entering my own studio at tim...,tv news anchor arnab goswami ha said he wa tol...
...,...,...,...
55099,55099,sensex loses point to hit week low,tracking weak cue from the asian ets the benc...
55100,55100,china to inject bn into the money ets,amid growing concern about china s economi...
55101,55101,ghulam ali set to make acting debut in bollywood,pakistani ghazal singer ghulam ali will soon m...
55102,55102,is acknowledges death of jihadi john report,the islamic state is ha acknowledged the dea...


In [ ]:
from sklearn.utils import shuffle
# 데이터 섞기 
data = shuffle(data)
data.head()

,Unnamed: 0,Headline,Short
19972,19972,kohli average in inning at vizag,virat kohli average in five inning at the apc...
44018,44018,jawan killed in fire at army depot in maha,as many a defence security corps dsc jawan ...
16117,16117,hyperloop to reduce punemumbai travel time to ...,usbased highspeed transportation systthem hype...
8626,8626,universe is expanding up to faster than expe...,a new study using nasa s hubble space tele...
30235,30235,indiaborn scientist win crore mit award,indiaborn scientist ramesh raskar wa awarded ...


In [ ]:
data.drop(['Source ', 'Time ', 'Publish Date'], axis=1, inplace=True) #떨궈서 필요한 거만 남기기, inplay=True -> 원본에 그대로 적용
data

In [ ]:
data.shape 

In [ ]:
article = data['Short']
summary = data['Headline']

In [ ]:
article[25]

In [ ]:
 summary[25]

# 텍스트 전처리 
## 여러 조건을 바꿔가며 실험해 보자

In [ ]:
!pip install -q contractions==0.0.48

In [ ]:
from contractions import contractions_dict # 아포트로스피등을 사용한 축약문을 바꿀 수 있도록 정리한 단어사전

for key, value in list(contractions_dict.items())[:15]: # contraction_dict에서 15개 예시를 출력해보자.
    print(f'{key} == {value}')

In [ ]:
def expand_contractions(text, contraction_map=contractions_dict):
    # 정규표현식을 사용해서 축약된 단어들을 모두 찾아와서 바꾸기
    contractions_keys = '|'.join(contraction_map.keys()) # contractio_map의 key에 '|'를 추가
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL) # contraction_pattern에 정규표현식 형태를 저장 ; 컴파일 옵션으로 . 메타 문자가 줄바꿈 문자(\n)를 포함하여 모든 문자와 매치되도록함

    def expand_match(contraction):# 축약문에 맞는 문자열을 찾는 함수
        # 조건에 맞는 전체 하위 문자열을 가져오기
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text) # 축약문을 해당하는 문자열로 변환
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text 

# 함수가 잘 작동하는지 테스트 해보자
expand_contractions("we'll gonna to expand contractions i'd think")

In [ ]:
# 앞에서 만들어 두었던 expand_contractions 함수를 적용
article = article.apply(expand_contractions)
summary = summary.apply(expand_contractions)

In [ ]:
#테스트 해보자
data.head(10)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #불용어 사전 다운로드

In [ ]:
# nltk라이브러리에서 불용어 리스트를 불러워서 텍스트내의 불용어들을 제거하는 함수 작성
def delete_stopwords(text):
    _stopwords = stopwords.words('english') 
    _stopwords.append('cnn') # 불용어 사전에 (cnn)을 추가함. 이는 문서의 의미에 영향을 끼치지 않을 것이기 때문. 괄호지우는 작업도 정규표현식으로 더했더니 cnn만 남아서 그냥 cnn은 다 지워 버리는걸로 결정함
    text = text.split()
    word_list = [word for word in text if word not in _stopwords]
    return ' '.join(word_list)

# 함수를 테스트 해보자
remove_stopwords_from_text("Love means never having to say you're sorry") # 의미가 있는 단어 having이 통채로 사라져버렸다!! 적용하지 않는게 좋을 듯 하다.

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')  #lemmatize를 위한 단어사전

In [ ]:
# 표제어추출을 위한 함수 작성
def lemmatize_text(data):
    data = nltk.word_tokenize(data)
    lemtzr=WordNetLemmatizer()
    out_data=" "
    for words in data:
        out_data+= lemtzr.lemmatize(words)+" "
    return out_data

#함수 테스트 해보자
lemmatize_text("she goes to the hospitals") # 동사에 붙는 s, 복수에 붙는 s 등이 사라지는 것을 확인할 수 있다! # 이를 통해 단어 사전을 줄임으로서 문장의 의미를 최대한 유지하면서 연산량을 줄일 수 있다.

In [ ]:
#표제어 추출 실행
article=article.apply(lemmatize_text) # 원문에는 적용하지 않는 방법도 시도해 보자 # 아! 서비스에는 raw-senctence가 들어오니까 적용하면 안된다!!, 혹은 평가단계에서도 적용하여야 한다.
summary=summary.apply(lemmatize_text) # summary에만 적용하자

In [ ]:
article[25]

In [ ]:
summary[25]

In [ ]:
#영어는 길이가 짧은 단어는 의미가 없다고 함
#그런데 us와 같은 단어는 길이가 짧지만 경우에 따라서 큰 의미를 지니고 있을수도 있다고 생각이 들었음.
#해서 길이가 짧은 단어를 지울지 말지는 고민해보고, 성능비교를 할때 적용해서 비교해보든지 해야겠다고 생각하였음.
#길이가 짧은 단어를 제거하려면 다음과 같이 하면된다.
#data.highlights=data.highlights.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#data.article=data.article.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
# 디코더의 작동을 위해 문장의 시작과 끝을 표시
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary.head()

In [ ]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' #정규표현식으로 필터 정의
oov_token = '<unk>' # 단어장에 출현하지 않은, 모르는 단어를 표시할 토큰

In [ ]:
# 케라스로 텍스트 토크나이징하는 토크나이징 메소드 생성
article_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token) # 필터 적용

In [ ]:
# 데이터에 메소드 적용
article_tokenizer.fit_on_texts(article)
summary_tokenizer.fit_on_texts(summary)

In [ ]:
# 정수 시퀀스로 변경하고 입력시퀀스와 타깃 시퀀스로 지정
inputs = article_tokenizer.texts_to_sequences(article)
targets = summary_tokenizer.texts_to_sequences(summary)

In [ ]:
# 확인해보자
summary_tokenizer.texts_to_sequences(["I'm a bad guy"])

In [ ]:
summary_tokenizer.sequences_to_texts([[1, 10, 1227, 4955]])

In [ ]:
#그렇다면 contractions를 적용한 문장은?
summary_tokenizer.texts_to_sequences(["I would like to go home"])

In [ ]:
# unk 토큰없이 아주 잘된다.
summary_tokenizer.sequences_to_texts([[54, 467, 257, 4, 149, 220]])

In [ ]:
# 단어사전의 크기를 각각 지정
encoder_vocab_size = len(article_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# 단어사전의 크기 확인
encoder_vocab_size, decoder_vocab_size # 전처리를 까먹으면 단어사전 크기가 (76362, 29661) 가 나온다

In [ ]:
article_lengths = pd.Series([len(x) for x in article])
summary_lengths = pd.Series([len(x) for x in summary])

In [ ]:
article_lengths.describe()

In [ ]:
summary_lengths.describe()

In [ ]:
# 정수 시퀀스의 최대 길이를 지정
encoder_maxlen = 420
decoder_maxlen = 80

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
inputs, inputs_val, targets, targets_val = train_test_split(inputs, targets, test_size=0.2, random_state=17)

In [ ]:
# padding과 truncating  ; 이걸 안해서 문제가 생겼던 건 아닐까
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post') 
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')
inputs_val=tf.keras.preprocessing.sequence.pad_sequences(inputs_val,maxlen=encoder_maxlen,padding='post',truncating='post')
targets_val=tf.keras.preprocessing.sequence.pad_sequences(targets_val,maxlen=decoder_maxlen,padding="post",truncating='post')

In [ ]:
# 텐서를 int32 타입으로 캐스팅
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)
inputs_val=tf.cast(inputs_val,dtype=tf.int32)
targets_val=tf.cast(targets_val,dtype=tf.int32)

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
#텐서를 사용하여 데이터셋을 초기화 ; 

#The given tensors are sliced along their first dimension. This operation preserves the structure of the input tensors, 
#removing the first dimension of each tensor and using it as the dataset dimension. All input tensors must have the same size in their first dimensions.
#This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements.
#For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.
#For instance, if your dataset contains 10,000 elements but buffer_size is set to 1,000, then shuffle will initially select a random element from only the first 1,000 elements in the buffer.
#Once an element is selected, its space in the buffer is replaced by the next (i.e. 1,001-st) element, maintaining the 1,000 element buffer.

dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset_val=tf.data.Dataset.from_tensor_slices((inputs_val,targets_val)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE*2)

# 단어에 위치 개념을 추가하기 위한 함수 정의 ;

In [ ]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # 짝수 인덱스는 sin
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # 홀수 인덱스는 cos
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

# Attn 함수들 ; Attention is all you need github에서 가져옴

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True) # transposed key와 행렬곱

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)  # normalize를 위해 차원수로 나눠주기

    if mask is not None: # 마스킹할 영역 준비, mask가 있으면
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) #소프트맥스에 통과시켜 확률값으로 변환

    output = tf.matmul(attention_weights, v) # value와 행렬곱
    return output, attention_weights # 컨텍스트 벡터를 리턴

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # 몇개로 나눠줄 것인가
        self.d_model = d_model # 임베딩 차원수

        assert d_model % self.num_heads == 0 

        self.depth = d_model // self.num_heads #헤드 갯수로 나주어준다

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model) 
        
    def split_heads(self, x, batch_size): # 헤드 나누기
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)  # 각각 linear dense에 통과시킨후

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size) #배치사이즈에 따라 나눈다.

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask) #스케일드 닷 어텐션

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model)) # 원래 모양으로 돌려주기 위해
        output = self.dense(concat_attention) # concat
            
        return output, attention_weights

In [ ]:
#피드포워드 네트워크
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([ #레이어 순차적으로
        tf.keras.layers.Dense(dff, activation='relu'), # 뉴럴 수, relu
        tf.keras.layers.Dense(d_model) # 임베딩 차원
    ])

In [ ]:
#기초적인 인코더레이어 유닛
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads) # 멀티헤드 어텐션
        self.ffn = point_wise_feed_forward_network(d_model, dff) # 피드포워드

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6) # 노멀라이즈
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6) #노멀라이즈

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate) #드롭아웃
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask) 
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


In [ ]:
#기초적인 디코더레이어 유닛
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


In [ ]:
#아까 정의한 인코더 레이어를 여러장 겹쳐서 인코더를 정의
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1): # d_model 워드 임베딩의 차원 ,num_layer: 적층할 레이어 갯수, dff: 피드포워드 네트워크의 뉴런갯수
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model) # 워드 임베딩을 전달
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model) #포지션 인코딩 

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)] #num_layer는 4장, 4장의 인코더 레이어로 구성

        self.dropout = tf.keras.layers.Dropout(rate) # dropout 설정
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x) 
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


In [ ]:
#마찬가지로 디코더 레이어를 여러장 겹쳐서 디코더 정의
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model) # 임베딩 레이어
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model) #포지션 인코딩

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)] 
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


In [ ]:
# 인코더 디코더를 합쳐서 트랜스포머 정의
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate) #인코더

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate) # 디코더

        self.final_layer = tf.keras.layers.Dense(target_vocab_size) #Dens 레이어로 마무리
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask) 

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


# 모델 훈련 단계


In [ ]:
# 하이퍼 파라미터 조정
num_layers = 4 #
d_model = 128 # embedding dim
dff = 512 #피드 포워드에 집어넣을 뉴런 수
num_heads = 8 #
EPOCHS = 37 #

In [ ]:
#커스텀 러닝 스케쥴 사용
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [ ]:
# 러닝 레이트와 옵티마이저 설정
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [ ]:
# loss metric 설정
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_loss = tf.keras.metrics.Mean(name='val_loss')

In [ ]:
transformer = Transformer(num_layers,d_model,num_heads,dff,encoder_vocab_size,decoder_vocab_size,pe_input=encoder_vocab_size,pe_target=decoder_vocab_size,)

In [ ]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


In [ ]:
# 훈련 체크포인트 설정
checkpoint_path = "checkpoints_01" # 폴더 경로

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer) 

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('체크포인트 복구됨')

In [ ]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
history={'val':[],'train':[]}

train_loss = tf.keras.metrics.Mean(name='train_loss')
val_loss = tf.keras.metrics.Mean(name='val_loss')

In [ ]:
def hist(history):
    plt.title('Loss')

    x= [i[0] for i in history['val']]
    y=[i[1] for i in history['val']]
    plt.plot(x,y,'x-')
    
    x= [i[0] for i in history['train']]
    y=[i[1] for i in history['train']]    
    plt.plot(x,y,'o-')

    plt.legend(['validation','train'])
    plt.show()
    print('smallest val loss:', sorted(history['val'],key=lambda x: x[1])[0])

In [ ]:
EPOCHS = 37
not_progressing = 0
# Computes the (weighted) mean of the given loss values.


In [ ]:
def validate():
    print('validation started ...')
    val_loss.reset_states()
    for (batch, (inp, tar)) in enumerate(dataset_val):    
        tar_inp = tar[:, :-1] 
        tar_real = tar[:, 1:] 

        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

        predictions, _ = transformer(inp, tar_inp, False,enc_padding_mask,combined_mask,dec_padding_mask)
        loss = loss_function(tar_real, predictions)
        val_loss(loss)
    print('\n* Validation loss: {} '.format(val_loss.result()) )
    return val_loss.result()


In [ ]:
#훈련 단계
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
            val_loss_ = validate().numpy()
            history['val'].append((epoch,val_loss_))
            print ('\n* Train Loss {:.4f}'.format(train_loss.result()))
            history['train'].append((epoch,train_loss.result().numpy())) 
    
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))
    hist(history)
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


# 평가단계


In [ ]:
# 디코더에서 한 번에 한 단어씩 예측하여 출력에 추가한 다음 
# 전체 시퀀스를 디코더의 입력으로 가져와서 문장의 끝이 나타날 때까지 반복
def evaluate(input_document):
    input_document = article_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index["<go>"]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index["<stop>"]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [ ]:
# 입력을 요약하는 함수
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

In [ ]:
# 테스트 해보자
summarize(
    "US-based private equity firm General Atlantic is in talks to invest about \
    $850 million to $950 million in Reliance Industries' digital unit Jio \
    Platforms, the Bloomberg reported. Saudi Arabia's $320 billion sovereign \
    wealth fund is reportedly also exploring a potential investment in the \
    Mukesh Ambani-led company. The 'Public Investment Fund' is looking to \
    acquire a minority stake in Jio Platforms."
)

# Rouge metric

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 


In [ ]:
hypothesis = "the transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"
#추측
reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

#원래 요약문
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
scores

In [ ]:
# 원문은 article[21], A police officer in Lucknow, Nagesh Mishra, was suspended from duty after he made everyone swear to a vow of cleanliness while he was himself consuming tobacco. Notably,
#Uttar Pradesh CM Yogi Adityanath has directed officials against consuming pan masala in office premises.
#Over 100 Uttar Pradesh cops have been suspended over discipline-related issues ever since Adityanath assumed office.

hypothesis=summarize("A police officer in Lucknow, Nagesh Mishra, was suspended from duty after he made everyone swear to a vow of cleanliness while he was himself consuming tobacco. Notably,Uttar Pradesh CM Yogi Adityanath has directed officials against consuming pan masala in office premises.Over 100 Uttar Pradesh cops have been suspended over discipline-related issues ever since Adityanath assumed office.")

reference="UP cop suspended for consuming tobacco"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference,avg=True)
scores

In [ ]:
article[300], summary[300]

In [ ]:
summarize("A police officer in Lucknow, Nagesh Mishra, was suspended from duty after he made everyone swear to a vow of cleanliness while he was himself consuming tobacco. Notably,Uttar Pradesh CM Yogi Adityanath has directed officials against consuming pan masala in office premises.Over 100 Uttar Pradesh cops have been suspended over discipline-related issues ever since Adityanath assumed office.")

In [ ]:
summary[40000]

In [ ]:
for i in range (len(article)):
  _hypothesis=summarize(article[i])
  _reference=summary[i]

  rouge=Rouge()
  scores=rouge.get_scores(_hypothesis,_reference,avg=True)
  print(scores)


In [ ]:
# 원문은 article[21], A police officer in Lucknow, Nagesh Mishra, was suspended from duty after he made everyone swear to a vow of cleanliness while he was himself consuming tobacco. Notably,
#Uttar Pradesh CM Yogi Adityanath has directed officials against consuming pan masala in office premises.
#Over 100 Uttar Pradesh cops have been suspended over discipline-related issues ever since Adityanath assumed office.

hypothesis=summarize("A police officer in Lucknow, Nagesh Mishra, was suspended from duty after he made everyone swear to a vow of cleanliness while he was himself consuming tobacco. Notably,Uttar Pradesh CM Yogi Adityanath has directed officials against consuming pan masala in office premises.Over 100 Uttar Pradesh cops have been suspended over discipline-related issues ever since Adityanath assumed office.")

reference="UP cop suspended for consuming tobacco"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference,avg=True)
scores

In [ ]:
for i in range (len(article)):
  _hypothesis=_hypothesis.append(summarize(article[i]))
  _reference=_reference.append(summary[i])

  rouge=Rouge()
  scores=rouge.get_scores(_hypothesis,_reference,avg=True)
  print(scores)


# 시험


In [ ]:
summarize("A victory for one of the hardliners is not expected to derail talks in Vienna between Iran and world powers that are aimed at reviving the accord, which saw Iran agree to limit its nuclear programme in return for sanctions relief.")

# ToDo
